In [ ]:
%pip install -r requirements.txt

### Load Dataset

The hugging face `datasets` library lets you manipulate data from the HF hub (https://huggingface.co/datasets).

The ultrachat dataset https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k has examples of a multi-turn chat.

In [27]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
ds[0]["messages"]

[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
  'role': 'user'},
 {'content': "Sure, here are

### Tokenization

A tokenizer takes text and converts it to the `input_ids` that we pass to the model. In the case below, I downloaded the tokenizer for Zephyr (which is a fine-tuned variant of Mistral).

Recently, tokenizers added a feature called `apply_chat_template` that takes a string of messages and gives back a formatted chat string. The models are trained with this chat format during the "alignment" phase. See more (https://huggingface.co/HuggingFaceH4/zephyr-7b-beta).

Note: all of this chat formatting is handled on the vLLM side. This is just here as an FYI.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
tokenizer("Hello my name is")

{'input_ids': [1, 22557, 586, 1141, 349], 'attention_mask': [1, 1, 1, 1, 1]}

In [17]:
# With a chat template

print("---- input ids corresponding to the string:")
print(tokenizer.apply_chat_template(ds[0]["messages"])) 
print("\n---- actual text:")
print(tokenizer.apply_chat_template(ds[0]["messages"], tokenize=False))

---- input ids corresponding to the string:
[523, 28766, 1838, 28766, 28767, 13, 18171, 11382, 5580, 298, 4211, 28733, 5527, 18978, 325, 1146, 13532, 495, 28705, 28784, 28723, 28734, 28806, 28725, 8337, 1380, 28705, 28781, 28723, 28734, 28806, 28725, 2316, 455, 897, 28705, 28770, 28723, 28734, 28806, 6372, 1798, 28705, 28750, 28723, 28734, 28806, 28725, 351, 598, 16712, 28705, 28782, 28723, 28734, 28806, 609, 1824, 7335, 2751, 837, 315, 1413, 28804, 13, 2486, 574, 27395, 6718, 567, 22114, 28715, 27395, 12458, 28725, 368, 541, 5061, 1347, 272, 13461, 3469, 302, 264, 2093, 356, 18848, 486, 25748, 624, 302, 272, 7335, 28742, 28713, 4429, 28733, 262, 6472, 28808, 13, 11159, 13079, 6718, 567, 22114, 28715, 27395, 12458, 622, 1055, 4249, 272, 13461, 2093, 3469, 776, 486, 18848, 288, 754, 369, 2093, 3469, 15762, 21418, 28723, 13, 20510, 456, 4480, 5580, 298, 544, 12458, 302, 272, 7335, 442, 776, 2948, 4413, 390, 9206, 297, 272, 2245, 3388, 28804, 2, 28705, 13, 28789, 28766, 489, 11143, 28766,

In [18]:
# Only send the first N turns
N = 3

print(tokenizer.apply_chat_template(ds[0]["messages"][:N], tokenize=False))

<|user|>
These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?</s>
<|assistant|>
This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.</s>
<|user|>
Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?</s>



[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
  'role': 'user'}]

In [25]:
# Add a system prompt. Only send the first N turns
N = 3

system_message = {
    "content": "You are a chatbot with the explict goal of helping the user as best as possible",
    "role": "system",
}

messages = [system_message]
messages.extend(ds[0]["messages"][:N])
messages
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|system|>
You are a chatbot with the explict goal of helping the user as best as possible</s>
<|user|>
These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?</s>
<|assistant|>
This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.</s>
<|user|>
Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?</s>



### vLLM OpenAI Server

vLLM uses the openai protocol. Spin up a vllm server with:

```bash
python3 -m vllm.entrypoints.openai.api_server \
    --model HuggingFaceH4/zephyr-7b-beta \
    --max-model-len 2048 \
    --tensor-parallel-size 1 \
    --disable-log-requests \
    --enforce-eager
```

In [38]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# Get the model name from the api server
models = client.models.list()
model = models.data[0].id
print(model)

APIConnectionError: Connection error.

In [ ]:
# 
chat_completion = client.chat.completions.create(
    messages=[{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": "Who won the world series in 2020?"
    }, {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020."
    }, {
        "role": "user",
        "content": "Where was it played?"
    }],
    model=model,
)

print("Chat completion results:")
print(chat_completion)

In [31]:
# Add a system prompt. Only send the first N turns
system_message = {
    "content": "You are a chatbot with the explict goal of helping the user as best as possible",
    "role": "system",
}

messages = [system_message]
messages.extend(ds[0]["messages"])
print(len(messages))

9


In [35]:
first_request = messages[:-3]
first_request

[{'content': 'You are a chatbot with the explict goal of helping the user as best as possible',
  'role': 'system'},
 {'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hove

In [37]:
second_request = messages[:-1]
second_request

[{'content': 'You are a chatbot with the explict goal of helping the user as best as possible',
  'role': 'system'},
 {'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hove